# Sentence Dependencies

Ok, so a big part in getting better scores will be to narrow down the problem of the sentence pairs, for this we will need to know important words and their dependents. This notebook will serve solely to work out methods how to get those, so the other ones are not overloaded with stuff (like First Steps).

In [23]:
import spacy
import numpy as np
import pandas as pd

parser = spacy.load('en_core_web_sm')

test_a = 'I put a elephant in the fridge'
test_b = 'I put a turkey in the fridge'

# for this we want to remove stop words
# !!! We also want to know the index of the different words, HAS TO BE IMPLEMENTED!!!
different_words = ['elephant', 'turkey']
different_words_idx = [3,4]

parse_a = parser(test_a)
parse_b = parser(test_b)
print(parse_a)
print(parse_b)
for token in parse_a:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
print('\n\n')
for token in parse_b:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

I put a elephant in the fridge
I put a turkey in the fridge
I nsubj put VERB []
put ROOT put VERB [I, elephant, in]
a det elephant NOUN []
elephant dobj put VERB [a]
in prep put VERB [fridge]
the det fridge NOUN []
fridge pobj in ADP [the]



I nsubj put VERB []
put ROOT put VERB [I, turkey, in]
a det turkey NOUN []
turkey dobj put VERB [a]
in prep put VERB [fridge]
the det fridge NOUN []
fridge pobj in ADP [the]


In [33]:
""" THis will be methof get_dependents"""

"""PYTHONIC: maybe it would be cleaner to only do one sentence at a time,
and handle the structure of the sentences further up in the logic"""

#print(type(parse_a))
for w, idx in zip(different_words, different_words_idx):
    w = parser(w).text
    
    if w in parse_a.text:
        parent = parse_a[idx].head
        print(parent)
        
        # first we check what pos tags our parent has, in order to build a decision tree...
        if parent.pos_ == 'VERB':
            # now we want all children of the verb with subj and obj labels
            verb_dependents_a = []
            for child in parent.children:
                if child.dep_[-2:] == 'bj':
                    verb_dependents_a.append(child)
                else:
                    for nephew in child.children:
                        if nephew.dep_[-2:] == 'bj':
                            verb_dependents_a.append(nephew)
                            
    elif w in parse_b.text:
        parent = parse_b[idx].head
        
        # first we check what pos tags our parent has, in order to build a decision tree...
        if parent.pos_ == 'VERB':
            # now we want all children of the verb with subj and obj labels
            verb_dependents_b = []
            for child in parent.children:
                if child.dep_[-2:] == 'bj':
                    verb_dependents_b.append(child)
                else:
                    for nephew in child.children:
                        if nephew.dep_[-2:] == 'bj':
                            verb_dependents_b.append(nephew)

put


In [34]:
print(verb_dependents_a)
print(verb_dependents_b)

[I, elephant, fridge]
[I, turkey, fridge]


In [41]:
"""This will be method compute_similarity"""
#similarityy of two tokens ex:
# apples_oranges = apples.similarity(oranges)
dependents = verb_dependents_b
similarity = 0
cur, prev = None, None
for w in dependents:
    cur = w
    print(cur)
    if prev == None:
        prev = cur
        continue
    print(prev)
    similarity += prev.similarity(cur)
    print(similarity)
    prev = cur
    

I
turkey
I
0.10982343554496765
fridge
turkey
0.619660884141922


/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with

In [40]:
similarity_a = similarity
similarity_a

0.49096861109137535

In [42]:
similarity_b = similarity
similarity_b

0.619660884141922